# Process data

This notebook contains functionality for processing sensor data in order to 1) compute similarity between colocated and non-colocated devices, including different types of attack: injection (silent- or moving-baseline), replay (replay without delta compensation), and similar context (powerful adversary), 2) generate binary fingerprints from sensor data, and 3) calculate activity recognition metrics (i.e., activity filter in the paper) on chunks of sensor data.  

Some magic command to automatically reload changed modules

In [ ]:
%load_ext autoreload
%autoreload 2

Import necessary modules:

In [ ]:
from process.evaldata import eval_per_sensor, eval_per_sensor_pair
from const.globconst import *
from const.fpsconst import *
from common.util import get_sensor_data

### General HowTo: 
### 1) load sensor data for the specified experiment: in cells 'AR' to 'Compute key corpus' and 'Replay' only *one* experiment should be provided, in the 'Baseline' cells—*two*.

### 2) provide necessary parameters: in cells 'AR' to 'Compute key corpus' it is *modality* (e.g., 'acc_v'), while in 'Baseline' and 'Replay' cells, these are *sensor pairs* for computing.

### 3) cells are executed one after another, i.e., first necessary parameters are set and computing begins

### See comments in individual cells for more details.

Specify experiments:

* **SIM_NON_ADV** - similar cars, non-adversarial setup (all scenarios)
* **SIM_ADV** - similar cars, adversarial setup (all scenarios)
* **DIFF_PARK** - different cars, parking scenario (non-adversarial setup)
* **DIFF_NON_ADV** - different cars, non-adversarial setup (all scenarios except parking)
* **DIFF_ADV** - different cars, adversarial setup (all scenarios)

In [ ]:
exp1 = DIFF_NON_ADV
exp2 = None         

Load and process data

Sensor names used below: 

* **acc_v** - vertical acceleration
* **acc_h** - horizontal acceleration
* **gyrW** - gyroscope sky-axis
* **bar** - altitude computed from barometric pressure

In [ ]:
acc, gyr, bar = get_sensor_data(exp1, exp2)

### Activity recognition (AR) evaluation

In [ ]:
# Adjust <EXPERIMENT> and <MODALITY> here (e.g., experiment = exp2, modality = 'gyrW')
experiment = exp1
modality = 'gyrW'

# Do not adjust these parameters
action = AR
tf = (0, -1)

In [ ]:
# Generate activity recognition (AR) logs

if modality == 'acc_v':
    # Data for CAR1 and CAR2 must be computed seprately, do NOT do [CAR1, CAR2]!
    eval_per_sensor(acc, 'acc_v', CAR1, tf, action, experiment)
    eval_per_sensor(acc, 'acc_v', CAR2, tf, action, experiment)
    
elif modality == 'acc_h':
    eval_per_sensor(acc, 'acc_h', CAR1, tf, action, experiment)
    eval_per_sensor(acc, 'acc_h', CAR2, tf, action, experiment)
    
elif modality == 'gyrW':
    eval_per_sensor(gyr, 'gyrW', CAR1, tf, action, experiment)
    eval_per_sensor(gyr, 'gyrW', CAR2, tf, action, experiment)
    
elif modality == 'bar':
    eval_per_sensor(bar, 'bar', CAR1, tf, action, experiment)
    eval_per_sensor(bar, 'bar', CAR2, tf, action, experiment)
else:
    print('Unknown modality %s' % (modality,))

### Generate log files for powerful adversary use case

In [ ]:
# Adjust <EXPERIMENT> and <MODALITY> here (e.g., experiment = exp2, modality = 'gyrW')
# Here 'experiment' can only be SIM_ADV or DIFF_ADV
experiment = exp1
modality = 'bar'

action = KEYS
# Set up the correct timeframe
if experiment == SIM_ADV:
    tf = (0, 11540)
elif experiment == DIFF_ADV:
    tf = (0, -1)
else:
    print('Unknown <experiment> %s: can only be SIM_ADV or DIFF_ADV!' % (experiment,))

In [ ]:
# Generate powerful adversary logs (Note: the resulting folder is named 'keys')

if modality == 'acc_v':
    # Data for CAR1 and CAR2 must be computed seprately, do NOT do [CAR1, CAR2]!
    eval_per_sensor(acc, 'acc_v', CAR1, tf, action, experiment, rnoise='chunk', powerful=True)
    eval_per_sensor(acc, 'acc_v', CAR2, tf, action, experiment, rnoise='chunk', powerful=True)
    
elif modality == 'acc_h':
    eval_per_sensor(acc, 'acc_h', CAR1, tf, action, experiment, rnoise='chunk', powerful=True)
    eval_per_sensor(acc, 'acc_h', CAR2, tf, action, experiment, rnoise='chunk', powerful=True)
    
elif modality == 'gyrW':
    eval_per_sensor(gyr, 'gyrW', CAR1, tf, action, experiment, rnoise='chunk', powerful=True)
    eval_per_sensor(gyr, 'gyrW', CAR2, tf, action, experiment, rnoise='chunk', powerful=True)
    
elif modality == 'bar':
    eval_per_sensor(bar, 'bar', CAR1, tf, action, experiment, rnoise='chunk', powerful=True)
    eval_per_sensor(bar, 'bar', CAR2, tf, action, experiment, rnoise='chunk', powerful=True)
else:
    print('Unknown modality %s' % (modality,))

### Benign evaluation (or xcorr):

In [ ]:
# Adjust <ACTION>, <EXPERIMENT> and <MODALITY> here (e.g., action='benign', experiment = exp2, modality = 'bar')
action = 'benign'
experiment = exp1
modality = 'bar'

# Set up timeframe depending on the experiment
if experiment == SIM_NON_ADV:
    tf = (1210, -1)
    
elif experiment == SIM_ADV:
    tf = (0, 11540)
    
elif experiment == DIFF_PARK:
    tf = (530, 2810)
    
elif experiment == DIFF_NON_ADV:
    tf = (905, -1)
    
elif experiment == DIFF_ADV:
    tf = (0, -1)
    
else:
    print('Unknown experiment %s' % (experiment,))

#### CAR1: 

In [ ]:
# Sensor sensor_pairs and tframes are fixed params
sensor_pairs = [('01', '02'), ('01', '03'), ('01', '04'), ('01', '05'), ('02', '03'), ('02', '04'), ('02', '05'), 
                ('03', '04'), ('03', '05'), ('04', '05')]
tframes = [(tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf)] 

if modality == 'acc_v':
    eval_per_sensor_pair(acc, 'acc_v', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
elif modality == 'acc_h':
    eval_per_sensor_pair(acc, 'acc_h', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
elif modality == 'gyrW':
    eval_per_sensor_pair(gyr, 'gyrW', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
elif modality == 'bar':
    eval_per_sensor_pair(bar, 'bar', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
else:
    print('Unknown modality %s' % (modality,))

#### CAR2

In [ ]:
# Sensor sensor_pairs and tframes are fixed params
sensor_pairs = [('06', '07'), ('06', '08'), ('06', '09'), ('06', '10'), ('07', '08'), ('07', '09'), ('07', '10'), 
                ('08', '09'), ('08', '10'), ('09', '10')]
tframes = [(tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf), (tf, tf)] 

if modality == 'acc_v':
    eval_per_sensor_pair(acc, 'acc_v', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
elif modality == 'acc_h':
    eval_per_sensor_pair(acc, 'acc_h', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
elif modality == 'gyrW':
    eval_per_sensor_pair(gyr, 'gyrW', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
elif modality == 'bar':
    eval_per_sensor_pair(bar, 'bar', sensor_pairs, tframes, action, experiment, rnoise='chunk')
    
else:
    print('Unknown modality %s' % (modality,))

### Compute extended corpus of the keys for CAR1 and CAR2

In [ ]:
# Adjust <EXPERIMENT>, <MODALITY>, and <DATA_SET> here (e.g., experiment = exp2, modality = 'gyrW', data_set = 'full')
experiment = exp1
modality = 'bar'
data_set = 'full'         # Can only be 'full' or 'reduced'

# Do not change this parameter
action = KEYS

# Set up timeframe depending on the experiment
if experiment == SIM_NON_ADV:
    tf = (1210, -1)
    
elif experiment == SIM_ADV:
    tf = (0, 11540)
    
elif experiment == DIFF_PARK:
    tf = (530, 2810)
    
elif experiment == DIFF_NON_ADV:
    tf = (905, -1)
    
elif experiment == DIFF_ADV:
    tf = (0, -1)

else:
    print('Unknown experiment %s' % (experiment,))

# Set up n_bits and sanity check 'data_set'
if data_set == 'full':
    if modality == 'acc_v' or modality == 'acc_h':
        n_bits = BITS_ACC
    elif modality == 'gyrW':
        n_bits = BITS_GYR
    elif modality == 'bar':
        n_bits = BITS_BAR
    else:
        print('Unknown modality %s' % (modality,))
elif data_set == 'reduced':
    if modality == 'acc_v' or modality == 'acc_h':
        n_bits = int(BITS_ACC / 2 + 1)
    elif modality == 'gyrW':
        n_bits = int(BITS_GYR / 2 + 1)
    elif modality == 'bar':
        n_bits = int(BITS_BAR / 2 + 1)
    else:
        print('Unknown modality %s' % (modality,))
else:
    print('Unknown <data_set> %s: can only be "full" or "reduced"!' % (data_set,))

In [ ]:
# Generate fingerprints
if modality == 'acc_v':
    eval_per_sensor(acc, 'acc_v', CAR1, tf, action, experiment, n_bits, rnoise='chunk')
    eval_per_sensor(acc, 'acc_v', CAR2, tf, action, experiment, n_bits, rnoise='chunk')

elif modality == 'acc_h':
    eval_per_sensor(acc, 'acc_h', CAR1, tf, action, experiment, n_bits, rnoise='chunk')
    eval_per_sensor(acc, 'acc_h', CAR2, tf, action, experiment, n_bits, rnoise='chunk')

elif modality == 'gyrW':
    eval_per_sensor(gyr, 'gyrW', CAR1, tf, action, experiment, n_bits, rnoise='chunk')
    eval_per_sensor(gyr, 'gyrW', CAR2, tf, action, experiment, n_bits, rnoise='chunk')

elif modality == 'bar':
    eval_per_sensor(bar, 'bar', CAR1, tf, action, experiment, n_bits, rnoise='chunk')
    eval_per_sensor(bar, 'bar', CAR2, tf, action, experiment, n_bits, rnoise='chunk')

else:
    print('Unknown modality %s' % (modality,))

### Baseline adversary (silent): 

The structure of evaluation is given below

* **SIM_NON_ADV**; data is loaded as *get_sensor_data(SIM_NON_ADV, exp2)*, where *exp2* is one of the following: 
    * SIM_NON_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    

* **SIM_ADV**; data is loaded as *get_sensor_data(SIM_ADV, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    

* **DIFF_PARK**; data is loaded as *get_sensor_data(DIFF_PARK, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    

* **DIFF_NON_ADV**; data is loaded as *get_sensor_data(DIFF_NON_ADV, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    

* **DIFF_ADV**; data is loaded as *get_sensor_data(DIFF_ADV, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * DIFF_PARK
    * DIFF_NON_ADV

In [ ]:
action = 'baseline'

# Check if we have intra- or inter-experiment case
if exp2 is None:
    # Intra-experiment case (consider experiment where silent periods exist)
    if exp1 == SIM_NON_ADV:
        tf1 = (1210, -1)
        tf2 = (0, 1210)

    elif exp1 == DIFF_PARK:
        tf1 = (530, 2810)
        tf2 = (0, 530)
    
    elif exp1 == DIFF_NON_ADV:
        tf1 = (905, -1)
        tf2 = (0, 905)
    
    else:
        print('In the baseline (silent, INTRA) experiment 1 can only be "%s", "%s" or "%s"' % 
              (SIM_NON_ADV, DIFF_PARK, DIFF_NON_ADV))
        
    # Construct experiment string
    exp_str = exp1 + '_' + 'silent' + '_' + exp1
    
    # Addition to the key
    kad = ''
        
else:
    # Inter-experiment case: check if we are not doing the redundant work
    if exp1 == exp2:
        print('Experiment names are the same in the baseline (silent) INTER, double check the provided experiments!')
    
    # tf1 is from exp1 (all experiments)
    if exp1 == SIM_NON_ADV:
        tf1 = (1210, -1)
        
    elif exp1 == SIM_ADV:
        tf1 = (0, 11540)

    elif exp1 == DIFF_PARK:
        tf1 = (530, 2810)

    elif exp1 == DIFF_NON_ADV:
        tf1 = (905, -1)

    elif exp1 == DIFF_ADV:
        tf1 = (0, -1)
        
    else:    
        print('Unknown experiment 1 %s' % (exp1,))
    
    # tf2 is from exp 2 (only silent periods) 
    if exp2 == SIM_NON_ADV:
        tf2 = (0, 1210)
    
    elif exp2 == DIFF_PARK:
        tf2 = (0, 530)
    
    elif exp2 == DIFF_NON_ADV:
        tf2 = (0, 905)
        
    else:
        print('In the baseline (silent, INTER) experiment 2 can only be "%s", "%s" or "%s"' % 
              (SIM_NON_ADV, DIFF_PARK, DIFF_NON_ADV))
    
    # Construct experiment string
    exp_str = exp1 + '_' + 'silent' + '_' + exp2
    
    # Addition to the key
    kad = 'x'

### Baseline adversary (moving):

The structure of evaluation is given below

* **SIM_NON_ADV**; data is loaded as *get_sensor_data(SIM_NON_ADV, exp2)*, where *exp2* is one of the following: 
    * SIM_NON_ADV
    * SIM_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    * DIFF_ADV
    

* **SIM_ADV**; data is loaded as *get_sensor_data(SIM_ADV, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * SIM_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    * DIFF_ADV
    

* **DIFF_PARK**; data is loaded as *get_sensor_data(DIFF_PARK, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * SIM_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    * DIFF_ADV
    

* **DIFF_NON_ADV**; data is loaded as *get_sensor_data(DIFF_NON_ADV, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * SIM_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    * DIFF_ADV
    

* **DIFF_ADV**; data is loaded as *get_sensor_data(DIFF_ADV, exp2)*, where *exp2* is one of the following:
    * SIM_NON_ADV
    * SIM_ADV
    * DIFF_PARK
    * DIFF_NON_ADV
    * DIFF_ADV

In [ ]:
action = 'baseline'

# Check if we have intra- or inter-experiment case
if exp2 == None:
    # Intra-experiment case
    if exp1 == SIM_NON_ADV:
        tf1 = (1210, -1)
        tf2 = (1210, -1)
    
    elif exp1 == SIM_ADV:
        tf1 = (0, 11540)
        tf2 = (0, 11540)
    
    elif exp1 == DIFF_PARK:
        tf1 = (530, 2810)
        tf2 = (530, 2810)
    
    elif exp1 == DIFF_NON_ADV:
        tf1 = (905, -1)
        tf2 = (905, -1)
    
    elif exp1 == DIFF_ADV:
        tf1 = (0, -1)
        tf2 = (0, -1)
        
    else:
        print('Unknown experiment 1 %s' % (exp1,))
    
    # Construct experiment string
    exp_str = exp1 + '_' + 'moving' + '_' + exp1
    
    # Addition to the key
    kad = ''
    
else:
    # Inter-experiment case: check if we are not doing the redundant work
    if exp1 == exp2:
        print('Experiment names are the same in the baseline (moving) INTER, double check the provided experiments!')
    
    # tf1 is from exp1 (all experiments)
    if exp1 == SIM_NON_ADV:
        tf1 = (1210, -1)
        
    elif exp1 == SIM_ADV:
        tf1 = (0, 11540)

    elif exp1 == DIFF_PARK:
        tf1 = (530, 2810)

    elif exp1 == DIFF_NON_ADV:
        tf1 = (905, -1)

    elif exp1 == DIFF_ADV:
        tf1 = (0, -1)
        
    else:    
        print('Unknown experiment 1 %s' % (exp1,))
        
    # tf2 is from exp 2 (all experiments)     
    if exp2 == SIM_NON_ADV:
        tf2 = (1210, -1)
        
    elif exp2 == SIM_ADV:
        tf2 = (0, 11540)

    elif exp2 == DIFF_PARK:
        tf2 = (530, 2810)

    elif exp2 == DIFF_NON_ADV:
        tf2 = (905, -1)

    elif exp2 == DIFF_ADV:
        tf2 = (0, -1)
        
    else:    
        print('Unknown experiment 2 %s' % (exp2,))
        
    # Construct experiment string
    exp_str = exp1 + '_' + 'moving' + '_' + exp2
    
    # Addition to the key
    kad = 'x'

### Computation for baseline (both silent and moving): computaion takes a while, so I highly recommend distributing it on different machines, i.e., each machine computes on two lists of sensor pairs (see below). The computation is already parallelized on a pair level (i.e., elements in the *sensor_pairsX* list), so throwing more than 10 cores at the problem won't make it faster. 

#### Pick a pair per node: 01-02 (nodes 01 and 11)

In [ ]:
sensor_pairs1 = [('01', kad + '01'), ('01', kad + '02'), ('01', kad + '03'), ('01', kad + '04'), ('01', kad + '05'), 
                ('01', kad + '06'), ('01', kad + '07'), ('01', kad + '08'), ('01', kad + '09'), ('01', kad + '10')]

sensor_pairs2 = [('02', kad + '01'), ('02', kad + '02'), ('02', kad + '03'), ('02', kad + '04'), ('02', kad + '05'),
                ('02', kad + '06'), ('02', kad + '07'), ('02', kad + '08'), ('02', kad + '09'), ('02', kad + '10')]

#### Pick a pair per node: 03-04 (nodes 02 and 12)

In [ ]:
sensor_pairs1 = [('03', kad + '01'), ('03', kad + '02'), ('03', kad + '03'), ('03', kad + '04'), ('03', kad + '05'), 
                ('03', kad + '06'), ('03', kad + '07'), ('03', kad + '08'), ('03', kad + '09'), ('03', kad + '10')]

sensor_pairs2 = [('04', kad + '01'), ('04', kad + '02'), ('04', kad + '03'), ('04', kad + '04'), ('04', kad + '05'), 
                ('04', kad + '06'), ('04', kad + '07'), ('04', kad + '08'), ('04', kad + '09'), ('04', kad + '10')]

#### Pick a pair per node: 05-06 (nodes 03 and 14)

In [ ]:
sensor_pairs1 = [('05', kad + '01'), ('05', kad + '02'), ('05', kad + '03'), ('05', kad + '04'), ('05', kad + '05'), 
                ('05', kad + '06'), ('05', kad + '07'), ('05', kad + '08'), ('05', kad + '09'), ('05', kad + '10')]

sensor_pairs2 = [('06', kad + '01'), ('06', kad + '02'), ('06', kad + '03'), ('06', kad + '04'), ('06', kad + '05'), 
                ('06', kad + '06'), ('06', kad + '07'), ('06', kad + '08'), ('06', kad + '09'), ('06', kad + '10')]

#### Pick a pair per node: 07-08 (nodes 04 and 16)

In [ ]:
sensor_pairs1 = [('07', kad + '01'), ('07', kad + '02'), ('07', kad + '03'), ('07', kad + '04'), ('07', kad + '05'), 
                ('07', kad + '06'), ('07', kad + '07'), ('07', kad + '08'), ('07', kad + '09'), ('07', kad + '10')]

sensor_pairs2 = [('08', kad + '01'), ('08', kad + '02'), ('08', kad + '03'), ('08', kad + '04'), ('08', kad + '05'), 
                ('08', kad + '06'), ('08', kad + '07'), ('08', kad + '08'), ('08', kad + '09'), ('08', kad + '10')]

#### Pick a pair per node: 09-10 (nodes 05 and 20)

In [ ]:
sensor_pairs1 = [('09', kad + '01'), ('09', kad + '02'), ('09', kad + '03'), ('09', kad + '04'), ('09', kad + '05'), 
                ('09', kad + '06'), ('09', kad + '07'), ('09', kad + '08'), ('09', kad + '09'), ('09', kad + '10')]

sensor_pairs2 = [('10', kad + '01'), ('10', kad + '02'), ('10', kad + '03'), ('10', kad + '04'), ('10', kad + '05'),
                ('10', kad + '06'), ('10', kad + '07'), ('10', kad + '08'), ('10', kad + '09'), ('10', kad + '10')]

#### Run evaluation (joint for silent and moving)

In [ ]:
# Resulting timeframes either from silent or moving
tframes = [(tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2), 
           (tf1, tf2), (tf1, tf2)] 

eval_per_sensor_pair(acc, 'acc_v', sensor_pairs1, tframes, action, exp_str, rnoise='chunk')
eval_per_sensor_pair(acc, 'acc_v', sensor_pairs2, tframes, action, exp_str, rnoise='chunk')

eval_per_sensor_pair(acc, 'acc_h', sensor_pairs1, tframes, action, exp_str, rnoise='chunk')
eval_per_sensor_pair(acc, 'acc_h', sensor_pairs2, tframes, action, exp_str, rnoise='chunk')

eval_per_sensor_pair(gyr, 'gyrW', sensor_pairs1, tframes, action, exp_str, rnoise='chunk')
eval_per_sensor_pair(gyr, 'gyrW', sensor_pairs2, tframes, action, exp_str, rnoise='chunk')

eval_per_sensor_pair(bar, 'bar', sensor_pairs1, tframes, action, exp_str, rnoise='chunk')
eval_per_sensor_pair(bar, 'bar', sensor_pairs2, tframes, action, exp_str, rnoise='chunk')

### Replaying adversary (without *delta* compensation):

In [ ]:
# Adjust <EXPERIMENT> here (e.g., experiment = exp2)
experiment = exp1
action = 'replay'

if experiment == SIM_NON_ADV:
    tf1 = (1210, -1)
    tf2 = (1210, -1)
    
elif experiment == SIM_ADV:
    tf1 = (0, 11540)
    tf2 = (0, 11540)
    
elif experiment == DIFF_PARK:
    tf1 = (530, 2810)
    tf2 = (530, 2810)
    
elif experiment == DIFF_NON_ADV:
    tf1 = (905, -1)
    tf2 = (905, -1)
    
elif experiment == DIFF_ADV:
    tf1 = (0, -1)
    tf2 = (0, -1)
    
else:
    print('Unknown experiment %s' % (experiment,))

### Computation for replay: computaion takes a while, so I highly recommend distributing it on different machines, i.e., each machine computes on two lists of sensor pairs (see below). The computation is already parallelized on a pair level (i.e., elements in the *sensor_pairsX* list), so throwing more than 5 cores at the problem won't make it faster. 

#### Pick a pair per node: 01-02 (nodes 01 and 11)

In [ ]:
sensor_pairs1 = [('01', '06'), ('01', '07'), ('01', '08'), ('01', '09'), ('01', '10')]
sensor_pairs2 = [('02', '06'), ('02', '07'), ('02', '08'), ('02', '09'), ('02', '10')]

#### Pick a pair per node: 03-04 (nodes 02 and 12)

In [ ]:
sensor_pairs1 = [('03', '06'), ('03', '07'), ('03', '08'), ('03', '09'), ('03', '10')]
sensor_pairs2 = [('04', '06'), ('04', '07'), ('04', '08'), ('04', '09'), ('04', '10')]

#### Pick a pair per node: 05-06 (nodes 03 and 14)

In [ ]:
sensor_pairs1 = [('05', '06'), ('05', '07'), ('05', '08'), ('05', '09'), ('05', '10')]
sensor_pairs2 = [('06', '01'), ('06', '02'), ('06', '03'), ('06', '04'), ('06', '05')]

#### Pick a pair per node: 07-08 (nodes 04 and 16)

In [ ]:
sensor_pairs1 = [('07', '01'), ('07', '02'), ('07', '03'), ('07', '04'), ('07', '05')]
sensor_pairs2 = [('08', '01'), ('08', '02'), ('08', '03'), ('08', '04'), ('08', '05')]

#### Pick a pair per node: 09-10 (nodes 05 and 20)

In [ ]:
sensor_pairs1 = [('09', '01'), ('09', '02'), ('09', '03'), ('09', '04'), ('09', '05')]
sensor_pairs2 = [('10', '01'), ('10', '02'), ('10', '03'), ('10', '04'), ('10', '05')]

#### Run evaluation

In [ ]:
# Resulting timeframes
tframes = [(tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2), (tf1, tf2)] 

eval_per_sensor_pair(acc, 'acc_v', sensor_pairs1, tframes, action, experiment, rnoise='chunk')
eval_per_sensor_pair(acc, 'acc_v', sensor_pairs2, tframes, action, experiment, rnoise='chunk')

eval_per_sensor_pair(acc, 'acc_h', sensor_pairs1, tframes, action, experiment, rnoise='chunk')
eval_per_sensor_pair(acc, 'acc_h', sensor_pairs2, tframes, action, experiment, rnoise='chunk')

eval_per_sensor_pair(gyr, 'gyrW', sensor_pairs1, tframes, action, experiment, rnoise='chunk')
eval_per_sensor_pair(gyr, 'gyrW', sensor_pairs2, tframes, action, experiment, rnoise='chunk')

eval_per_sensor_pair(bar, 'bar', sensor_pairs1, tframes, action, experiment, rnoise='chunk')
eval_per_sensor_pair(bar, 'bar', sensor_pairs2, tframes, action, experiment, rnoise='chunk')